# Best Location to Open Coffee Shop in Singapore
### Applied Data Science - Capstone Project by Ben Sung
### July 31, 2019

- Explore and cluster the neighborhoods in Singapore
- Extract data of districts/areas of Singapore through scrapping from internet 
- Perform analysis based on the data for the information in the district/areas
- Provide maps to visualize neighborhoods and their clustering for insights of the district/areas
- Conclude the best shop location to open in Singapore

In [3]:
# !pip install folium
# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
!pip install bs4
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

  Stored in directory: /home/dsxuser/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [147]:
# SSL certification
# ctx = ssl.create_default_context()
# ctx.check_hostname = False
# ctx.verify_mode = ssl.CERT_NONE

source = requests.get('http://www.citypopulation.de/Singapore-Regions.html').text
soup = BeautifulSoup(source, 'html5lib')

In [148]:
table = soup.find('table', id="ts")
table_rows = table.find_all('tr')

In [149]:
dataExtracted = []
for row in table_rows:
    dataExtracted.append([t.text.strip() for t in row.find_all('td')])

In [150]:
df = pd.DataFrame(dataExtracted)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 9 columns):
0    34 non-null object
1    34 non-null object
2    34 non-null object
3    34 non-null object
4    34 non-null object
5    34 non-null object
6    34 non-null object
7    34 non-null object
8    34 non-null object
dtypes: object(9)
memory usage: 2.5+ KB


In [151]:
df.head()

,0,1,2,3,4,5,6,7,8
0,None,None,None,None,None,None,None,None,None
1,,Ang Mo Kio,NE,"180,112","179,297","174,770","165,710",1340,→
2,,Bedok,E,"284,318","294,519","289,750","281,300",2160,→
3,,Bishan,C,"89,746","91,298","90,700","88,490",740,→
4,,Bukit Batok,W,"126,200","144,198","139,270","144,410",1100,→


In [152]:
df.rename(columns={0:"Drop1", 1:"Neighborhood", 2:"Drop2", 3:"Drop3", 4:"Drop4", 5:"Drop5", 6:"Population", 7:"Drop6", 8:"Drop7"}, inplace=True)
df.drop(['Drop1', 'Drop2', 'Drop3', 'Drop4', 'Drop5', 'Drop6', 'Drop7'], axis=1, inplace=True)
df.drop([0], axis=0)
df.drop([0], axis=0, inplace=True)
df

,Neighborhood,Population
1,Ang Mo Kio,"165,710"
2,Bedok,"281,300"
3,Bishan,"88,490"
4,Bukit Batok,"144,410"
5,Bukit Merah,"151,870"
6,Bukit Panjang,"140,820"
7,Bukit Timah,"77,280"
8,Changi,"2,080"
9,Choa Chu Kang,"187,510"
10,Clementi,"93,000"


In [153]:
df.sort_values(by=['Population'], ascending=False, inplace=True)
df

,Neighborhood,Population
22,Queenstown,"97,870"
10,Clementi,"93,000"
23,River Valley,"9,930"
3,Bishan,"88,490"
25,Sembawang,"87,380"
13,Jurong East,"81,180"
7,Bukit Timah,"77,280"
17,Newton,"7,640"
18,Novena,"48,950"
16,Marine Parade,"47,250"


In [154]:
# Get the Top 10 most populous neighborhoods
df.drop([22,10,23,3,25,13,7,17,18,16,31,19,29,28,8,5,20,4,6,24,32,27,11,15], axis=0, inplace=True)
df

,Neighborhood,Population
2,Bedok,"281,300"
14,Jurong West,"266,720"
30,Tampines,"257,110"
33,Woodlands,"252,530"
26,Sengkang,"240,640"
12,Hougang,"223,010"
34,Yishun,"214,940"
9,Choa Chu Kang,"187,510"
1,Ang Mo Kio,"165,710"
21,Punggol,"161,570"


In [162]:
# Reset the index
df = df.reset_index()
df.drop(['index'], axis=1, inplace=True)
df

,Neighborhood,Population
0,Bedok,"281,300"
1,Jurong West,"266,720"
2,Tampines,"257,110"
3,Woodlands,"252,530"
4,Sengkang,"240,640"
5,Hougang,"223,010"
6,Yishun,"214,940"
7,Choa Chu Kang,"187,510"
8,Ang Mo Kio,"165,710"
9,Punggol,"161,570"


In [163]:
df.to_csv('singaporeTop10.csv', index=False)

In [180]:
df = pd.read_csv('singaporeTop10.csv')

In [182]:
df.insert(2, "Latitude", 0)
df.insert(3, "Longitude", 0)
df

,Neighborhood,Population,Latitude,Longitude
0,Bedok,"281,300",0,0
1,Jurong West,"266,720",0,0
2,Tampines,"257,110",0,0
3,Woodlands,"252,530",0,0
4,Sengkang,"240,640",0,0
5,Hougang,"223,010",0,0
6,Yishun,"214,940",0,0
7,Choa Chu Kang,"187,510",0,0
8,Ang Mo Kio,"165,710",0,0
9,Punggol,"161,570",0,0


In [183]:
df.dtypes

Neighborhood    object
Population      object
Latitude         int64
Longitude        int64
dtype: object

In [198]:
# Get the Singapore latitude and longitude
geolocator = Nominatim(user_agent="ny_explorer")

loc = geolocator.geocode('Bedok, Singapore')
df.iloc[0, df.columns.get_loc('Latitude')] = loc.latitude
df.iloc[0, df.columns.get_loc('Longitude')] = loc.longitude

loc = geolocator.geocode('Jurong West, Singapore')
df.iloc[1, df.columns.get_loc('Latitude')] = loc.latitude
df.iloc[1, df.columns.get_loc('Longitude')] = loc.longitude

loc = geolocator.geocode('Tampines, Singapore')
df.iloc[2, df.columns.get_loc('Latitude')] = loc.latitude
df.iloc[2, df.columns.get_loc('Longitude')] = loc.longitude

loc = geolocator.geocode('Woodlands, Singapore')
df.iloc[3, df.columns.get_loc('Latitude')] = loc.latitude
df.iloc[3, df.columns.get_loc('Longitude')] = loc.longitude

loc = geolocator.geocode('Sengkang, Singapore')
df.iloc[4, df.columns.get_loc('Latitude')] = loc.latitude
df.iloc[4, df.columns.get_loc('Longitude')] = loc.longitude

loc = geolocator.geocode('Hougang, Singapore')
df.iloc[5, df.columns.get_loc('Latitude')] = loc.latitude
df.iloc[5, df.columns.get_loc('Longitude')] = loc.longitude

loc = geolocator.geocode('Yishun, Singapore')
df.iloc[6, df.columns.get_loc('Latitude')] = loc.latitude
df.iloc[6, df.columns.get_loc('Longitude')] = loc.longitude

loc = geolocator.geocode('Choa Chu Kang, Singapore')
df.iloc[7, df.columns.get_loc('Latitude')] = loc.latitude
df.iloc[7, df.columns.get_loc('Longitude')] = loc.longitude

loc = geolocator.geocode('Ang Mo Kio, Singapore')
df.iloc[8, df.columns.get_loc('Latitude')] = loc.latitude
df.iloc[8, df.columns.get_loc('Longitude')] = loc.longitude

loc = geolocator.geocode('Punggol, Singapore')
df.iloc[9, df.columns.get_loc('Latitude')] = loc.latitude
df.iloc[9, df.columns.get_loc('Longitude')] = loc.longitude

df

,Neighborhood,Population,Latitude,Longitude
0,Bedok,"281,300",1.323976,103.930216
1,Jurong West,"266,720",1.339636,103.707339
2,Tampines,"257,110",1.354653,103.943571
3,Woodlands,"252,530",1.436897,103.786216
4,Sengkang,"240,640",1.390949,103.895175
5,Hougang,"223,010",1.373360,103.886091
6,Yishun,"214,940",1.428136,103.833694
7,Choa Chu Kang,"187,510",1.389260,103.743728
8,Ang Mo Kio,"165,710",1.369842,103.846609
9,Punggol,"161,570",1.398033,103.907331


In [199]:
# Save to a file
df.to_csv('singaporeCoor.csv', index=False)

In [232]:
df = pd.read_csv('singaporeCoor.csv')

In [233]:
df

,Neighborhood,Population,Latitude,Longitude
0,Bedok,"281,300",1.323976,103.930216
1,Jurong West,"266,720",1.339636,103.707339
2,Tampines,"257,110",1.354653,103.943571
3,Woodlands,"252,530",1.436897,103.786216
4,Sengkang,"240,640",1.390949,103.895175
5,Hougang,"223,010",1.373360,103.886091
6,Yishun,"214,940",1.428136,103.833694
7,Choa Chu Kang,"187,510",1.389260,103.743728
8,Ang Mo Kio,"165,710",1.369842,103.846609
9,Punggol,"161,570",1.398033,103.907331


In [241]:
n_latitude = df.loc[df.index[0], 'Latitude'] # neighborhood latitude value
n_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value
print(df.loc[0,'Neighborhood'], ":", n_latitude, n_longitude)

Bedok : 1.3239765 103.930216


In [242]:
# Get the Singapore latitude and longitude
address = 'Singapore, Singapore'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.3408528, 103.878446863736.


In [243]:
# create map of Singapore using latitude and longitude values
map_Singapore = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
# for lat, lng, borough, dfSingapore in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
for lat, lng, borough, df in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['Neighborhood']):
    label = '{}, {}'.format(df, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Singapore)  
    
map_Singapore

### Define Foursquare Credentials and Version

In [244]:
CLIENT_ID = 'AD1LGMPNUFZ1S1ZQEWBQZFB1RDJ4XGWOGPIHZLJAZFHH31OP' # your Foursquare ID
CLIENT_SECRET = 'OGGOSAT5W1G5WOGKPNRLQJD4S5JUPRUV0WK24MALHZ2AKBLR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AD1LGMPNUFZ1S1ZQEWBQZFB1RDJ4XGWOGPIHZLJAZFHH31OP
CLIENT_SECRET:OGGOSAT5W1G5WOGKPNRLQJD4S5JUPRUV0WK24MALHZ2AKBLR


### Let's explore the first neighborhood in our dataframe

In [253]:
df = pd.read_csv('singaporeCoor.csv')

In [255]:
# Get the neighborhood's latitude and longitude values.
neighbourhood_latitude = df.loc[df.index[0], 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value
neighbourhood_name = "Bedok" # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Bedok are 1.3239765, 103.930216.


### Now, let's get the top 100 venues that are in the Bedok within a radius of 500 meters.

In [256]:
LIMIT = 100  # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AD1LGMPNUFZ1S1ZQEWBQZFB1RDJ4XGWOGPIHZLJAZFHH31OP&client_secret=OGGOSAT5W1G5WOGKPNRLQJD4S5JUPRUV0WK24MALHZ2AKBLR&v=20180605&ll=1.3239765,103.930216&radius=500&limit=100'

In [257]:
# Send the GET request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d41aef5787dba002b20adcc'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bedok',
  'headerFullLocation': 'Bedok, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 60,
  'suggestedBounds': {'ne': {'lat': 1.3284765045000044,
    'lng': 103.9347088019297},
   'sw': {'lat': 1.3194764954999953, 'lng': 103.9257231980703}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4dca91d952b1c2222a85852c',
       'name': 'Bedok Chwee Kueh 勿洛水粿',
       'location': {'address': '#01-19 Bedok Interchange Hawker Centre',
        'crossStreet': '207 New Upper Changi Rd',
        'lat': 1.3249029,
        'lng': 103.93025,
        'label

### All the information is in the items key. Let's borrow the get_category_type function from the Foursquare lab.

In [258]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Now we are ready to clean the json and structure it into a pandas dataframe.

In [259]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Bedok Chwee Kueh 勿洛水粿,Chinese Restaurant,1.324903,103.930250
1,Ya Kun Kaya Toast 亞坤,Coffee Shop,1.324095,103.929198
2,Duke Bakery,Bakery,1.324691,103.932514
3,FairPrice Fínest,Supermarket,1.324140,103.929260
4,Song Zhou Luo Bo Gao 松洲箩卜糕,Breakfast Spot,1.324836,103.930520


In [260]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

60 venues were returned by Foursquare.


# 2. Explore Neighbourhoods in Singapore
### Let's create a function to repeat the same process to all the neighbourhoods in Singapore

In [261]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighbourhood and create a new dataframe called *SingaporeVenues*.

In [262]:
SingaporeVenues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Bedok
Jurong West
Tampines
Woodlands
Sengkang
Hougang
Yishun
Choa Chu Kang
Ang Mo Kio
Punggol


### Let's check the size of the resulting dataframe

In [263]:
print(SingaporeVenues.shape)
SingaporeVenues.head()

(409, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bedok,1.323976,103.930216,Bedok Chwee Kueh 勿洛水粿,1.324903,103.930250,Chinese Restaurant
1,Bedok,1.323976,103.930216,Ya Kun Kaya Toast 亞坤,1.324095,103.929198,Coffee Shop
2,Bedok,1.323976,103.930216,Duke Bakery,1.324691,103.932514,Bakery
3,Bedok,1.323976,103.930216,FairPrice Fínest,1.324140,103.929260,Supermarket
4,Bedok,1.323976,103.930216,Song Zhou Luo Bo Gao 松洲箩卜糕,1.324836,103.930520,Breakfast Spot


In [265]:
SingaporeVenues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ang Mo Kio,49,49,49,49,49,49
Bedok,60,60,60,60,60,60
Choa Chu Kang,13,13,13,13,13,13
Hougang,29,29,29,29,29,29
Jurong West,59,59,59,59,59,59
Punggol,5,5,5,5,5,5
Sengkang,28,28,28,28,28,28
Tampines,78,78,78,78,78,78
Woodlands,47,47,47,47,47,47


#### Let's find out how many unique categories can be curated from all the returned venues

In [266]:
print('There are {} uniques categories.'.format(len(SingaporeVenues['Venue Category'].unique())))

There are 106 uniques categories.


## 3. Analyze Each Neighborhood

In [267]:
# one hot encoding
Singapore_onehot = pd.get_dummies(SingaporeVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Singapore_onehot['Neighborhood'] = SingaporeVenues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Singapore_onehot.columns[-1]] + list(Singapore_onehot.columns[:-1])
Singapore_onehot = Singapore_onehot[fixed_columns]

Singapore_onehot.head()

,Neighborhood,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Basketball Court,Beijing Restaurant,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Casino,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Department Store,Dessert Shop,Dim Sum Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Hardware Store,High School,Hobby Shop,Hong Kong Restaurant,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Karaoke Bar,Korean Restaurant,Lingerie Store,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Noodle House,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soup Place,Spa,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Wings Joint
0,Bedok,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bedok,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bedok,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bedok,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Bedok,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### And let's examine the new dataframe size.

In [268]:
Singapore_onehot.shape

(409, 107)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [270]:
Singapore_grouped = Singapore_onehot.groupby('Neighborhood').mean().reset_index()
Singapore_grouped

,Neighborhood,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Basketball Court,Beijing Restaurant,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Casino,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Department Store,Dessert Shop,Dim Sum Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Hardware Store,High School,Hobby Shop,Hong Kong Restaurant,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Karaoke Bar,Korean Restaurant,Lingerie Store,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Noodle House,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soup Place,Spa,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Wings Joint
0,Ang Mo Kio,0.000000,0.000000,0.020408,0.000000,0.000000,0.020408,0.020408,0.000000,0.000000,0.000000,0.000000,0.061224,0.020408,0.000000,0.000000,0.000000,0.020408,0.000000,0.020408,0.000000,0.020408,0.000000,0.102041,0.020408,0.000000,0.040816,0.00000,0.000000,0.000000,0.020408,0.040816,0.000000,0.000000,0.081633,0.000000,0.000000,0.000000,0.020408,0.00000,0.000000,0.000000,0.020408,0.020408,0.00000,0.020408,0.000000,0.0,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040816,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.020408,0.000000,0.020408,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.040816,0.000000,0.040816,0.020408,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040816,0.040816,0.00000,0.000000,0.000000,0.000000,0.020408,0.00000,0.000000,0.000000
1,Bedok,0.016667,0.016667,0.050000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.016667,0.016667,0.016667,0.016667,0.016667,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.016667,0.066667,0.000000,0.000000,0.016667,0.00000,0.000000,0.016667,0.000000,0.033333,0.000000,0.000000,0.050000,0.000000,0.016667,0.033333,0.016667,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.000000,0.050000,0.000000,0.016667,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.016667,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.050000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.033333,0.016667,0.00000,0.000000,0.016667,0.000000,0.016667,0.00000,0.000000,0.016667
2,Choa Chu Kang,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.230769,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0

In [271]:
# Let's confirm the new size
Singapore_grouped.shape

(10, 107)

#### Let's print each neighborhood along with the top 5 most common venues

In [273]:
num_top_venues = 5

for hood in Singapore_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Singapore_grouped[Singapore_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ang Mo Kio----
                 venue  freq
0          Coffee Shop  0.10
1           Food Court  0.08
2      Bubble Tea Shop  0.06
3  Japanese Restaurant  0.04
4         Dessert Shop  0.04


----Bedok----
                 venue  freq
0          Coffee Shop  0.07
1       Sandwich Place  0.05
2     Asian Restaurant  0.05
3  Japanese Restaurant  0.05
4           Food Court  0.05


----Choa Chu Kang----
                  venue  freq
0  Fast Food Restaurant  0.23
1    Salon / Barbershop  0.08
2                Casino  0.08
3            Playground  0.08
4           Coffee Shop  0.08


----Hougang----
            venue  freq
0      Food Court  0.10
1     Coffee Shop  0.10
2    Noodle House  0.07
3    Soccer Field  0.07
4  Sandwich Place  0.03


----Jurong West----
                  venue  freq
0   Japanese Restaurant  0.12
1      Asian Restaurant  0.10
2  Fast Food Restaurant  0.10
3          Dessert Shop  0.07
4    Chinese Restaurant  0.07


----Punggol----
                 venue  freq
0 

#### Let's put that into a *pandas* dataframe

In [274]:
# First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [277]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Singapore_grouped['Neighborhood']

for ind in np.arange(Singapore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Singapore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ang Mo Kio,Coffee Shop,Food Court,Bubble Tea Shop,Sushi Restaurant,Japanese Restaurant,Sandwich Place,Supermarket,Fast Food Restaurant,Dessert Shop,Seafood Restaurant
1,Bedok,Coffee Shop,Food Court,Asian Restaurant,Japanese Restaurant,Sandwich Place,Noodle House,Café,Fast Food Restaurant,Chinese Restaurant,Supermarket
2,Choa Chu Kang,Fast Food Restaurant,Coffee Shop,Playground,Sandwich Place,Lingerie Store,Salon / Barbershop,Casino,Food Court,Thai Restaurant,Park
3,Hougang,Coffee Shop,Food Court,Noodle House,Soccer Field,Breakfast Spot,Park,Department Store,Food,Food & Drink Shop,Market
4,Jurong West,Japanese Restaurant,Fast Food Restaurant,Asian Restaurant,Chinese Restaurant,Dessert Shop,Café,Park,Coffee Shop,Food Court,Gym / Fitness Center
5,Punggol,Bus Station,High School,Chinese Restaurant,Bus Stop,Wings Joint,Frozen Yogurt Shop,Electronics Store,Fast Food Restaurant,Food,Food & Drink Shop
6,Sengkang,Fast Food Restaurant,Chinese Restaurant,Food Court,Bakery,Coffee Shop,Café,Shopping Mall,Video Store,Food Stand,Metro Station
7,Tampines,Coffee Shop,Bakery,Café,Japanese Restaurant,Food Court,Fast Food Restaurant,Bubble Tea Shop,Shopping Mall,Sushi Restaurant,Chinese Restaurant
8,Woodlands,Japanese Restaurant,Chinese Restaurant,Café,Coffee Shop,Asian Restaurant,Clothing Store,Indian Restaurant,Frozen Yogurt Shop,Shopping Mall,Fast Food Restaurant
9,Yishun,Food Court,Chinese Restaurant,Coffee Shop,Supermarket,Hainan Restaurant,Noodle House,Fast Food Restaurant,Bus Line,Fried Chicken Joint,Ramen Restaurant
